In [1]:
from dotenv import load_dotenv
load_dotenv()  # loads variables from .env file

import os
print(os.getenv("LANGCHAIN_API_KEY"))  # test: should print your key (or None if not set)

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


In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

|The| color| of| the| sky| can| vary| depending| on| several| factors|,| including| the| time| of| day|,| weather| conditions|,| and| location|.| Typically|,| during| a| clear| day|,| the| sky| appears| blue| due| to| the| scattering| of| sunlight| by| the| Earth's| atmosphere|.| At| sunrise| and| sunset|,| the| sky| can| take| on| hues| of| orange|,| pink|,| and| purple| due| to| the| angle| of| the| sun| and| the| increased| distance| the| sunlight| travels| through| the| atmosphere|.| On| cloudy| or| over|cast| days|,| the| sky| may| appear| gray|.| In| other| conditions|,| such| as| during| a| storm| or| certain| atmospheric| phenomena|,| the| sky| can| display| a| range| of| other| colors|.||

In [4]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

|The| sky| appears| blue| during| the| day| due| to| a| phenomenon| called| Ray|leigh| scattering|,| where| shorter| blue| wavelengths| of| sunlight| are| scattered| in| all| directions| by| the| gases| and| particles| in| the| Earth's| atmosphere|.| However|,| the| color| of| the| sky| can| change| depending| on| various| factors|,| such| as| time| of| day|,| weather| conditions|,| and| pollution| levels|.| For| instance|,| during| sunrise| or| sunset|,| the| sky| may| appear| red|,| orange|,| or| pink|.| In| over|cast| conditions|,| it| can| look| gray|.||

In [5]:
chunks[1]

AIMessageChunk(content='The', additional_kwargs={}, response_metadata={}, id='run--63b9e8ca-07e2-48d4-a96a-274d8224e114')

In [ ]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4] + chunks[5]

AIMessageChunk(content='The sky appears blue', additional_kwargs={}, response_metadata={}, id='run--63b9e8ca-07e2-48d4-a96a-274d8224e114')